<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Settings" data-toc-modified-id="Settings-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Settings</a></span></li><li><span><a href="#Main-Function" data-toc-modified-id="Main-Function-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Main Function</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#scrape_tag(tag,-scrape_interval,-scrape_repeat_times)" data-toc-modified-id="scrape_tag(tag,-scrape_interval,-scrape_repeat_times)-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>scrape_tag(tag, scrape_interval, scrape_repeat_times)</a></span></li></ul></li></ul></li><li><span><a href="#Sub-Functions" data-toc-modified-id="Sub-Functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Sub Functions</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#get_top_posts_URLs(tag,-scrape_time)" data-toc-modified-id="get_top_posts_URLs(tag,-scrape_time)-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>get_top_posts_URLs(tag, scrape_time)</a></span></li><li><span><a href="#existing_URLs(tag,-_new_top_posts),-add_to_existing_URLs(tag,-new_top_posts)" data-toc-modified-id="existing_URLs(tag,-_new_top_posts),-add_to_existing_URLs(tag,-new_top_posts)-3.0.2"><span class="toc-item-num">3.0.2&nbsp;&nbsp;</span>existing_URLs(tag, _new_top_posts), add_to_existing_URLs(tag, new_top_posts)</a></span></li><li><span><a href="#get_post_info(new_top_posts,-tag,-scrape_time)" data-toc-modified-id="get_post_info(new_top_posts,-tag,-scrape_time)-3.0.3"><span class="toc-item-num">3.0.3&nbsp;&nbsp;</span>get_post_info(new_top_posts, tag, scrape_time)</a></span></li><li><span><a href="#download_image(URL,-tag,-scrape_time)" data-toc-modified-id="download_image(URL,-tag,-scrape_time)-3.0.4"><span class="toc-item-num">3.0.4&nbsp;&nbsp;</span>download_image(URL, tag, scrape_time)</a></span></li><li><span><a href="#get_profile_info(profile_URLs,-scrape_time)" data-toc-modified-id="get_profile_info(profile_URLs,-scrape_time)-3.0.5"><span class="toc-item-num">3.0.5&nbsp;&nbsp;</span>get_profile_info(profile_URLs, scrape_time)</a></span></li><li><span><a href="#write_info(tag,-scrape_time,-new_top_posts,-image_paths,-top_post_likes,-top_post_description,-profile_URLs,-profile_name,-profile_bio,-profile_follower)" data-toc-modified-id="write_info(tag,-scrape_time,-new_top_posts,-image_paths,-top_post_likes,-top_post_description,-profile_URLs,-profile_name,-profile_bio,-profile_follower)-3.0.6"><span class="toc-item-num">3.0.6&nbsp;&nbsp;</span>write_info(tag, scrape_time, new_top_posts, image_paths, top_post_likes, top_post_description, profile_URLs, profile_name, profile_bio, profile_follower)</a></span></li></ul></li></ul></li><li><span><a href="#Let's-start-our-Instagram-top-posts-scraping" data-toc-modified-id="Let's-start-our-Instagram-top-posts-scraping-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Let's start our Instagram top posts scraping</a></span><ul class="toc-item"><li><span><a href="#We-have-scarped-following-seven-tags" data-toc-modified-id="We-have-scarped-following-seven-tags-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>We have scarped following seven tags</a></span></li></ul></li></ul></div>

In [1]:
from selenium import webdriver
import re
import os
import datetime
import requests
import time
from bs4 import BeautifulSoup as bs
from bs4 import Tag
from bson.binary import Binary
import pandas as pd
from pymongo import MongoClient

# Settings

In [2]:
# Define the location of the webdriver, please download the webdriver for Chrome at:
# https://chromedriver.chromium.org/downloads
browser = webdriver.Chrome("./chromedriver")

# Main Function

### scrape_tag(tag, scrape_interval, scrape_repeat_times)

**Instruction:**  
scrape_tag() is the main function and will call other sub functions,  
so you only need to run this fuction to scrape the top posts on Instagram.  
<br/>
<br/> 
**What each argument does:**  
**tag:** what tag to scrape;  
**scrape_interval:** time interval (in seconds) between each scrape. Recommend to be 1800 (30min);  
**scrape_repeat_times:** for this tag, scrape how many times.  
<br/>
<br/> 
**Example:**
if you want to scrape the tag #corgi for three times and an interval of 1800 seconds between two scrapes, please run  
scrape_tag('corgi', 1800, 3)  
<br/>
<br/> 
**What does the function do:**  
1) Create a folder to store all the results of the web scraping;  
2) Based on the tag, scrape interval, and number of scrapes you defined, scrape the top posts on Instagram;  
3) Gather the following information of the top posts:  
    - download the image  
    - get the description of the post  
    - get the number of likes  
    - get the profile name of whom posted the post  
    - get the profile bio  
    - get the number of the profile's follower  
4) Store all the information into MongoDB database  

In [3]:
def scrape_tag(tag, scrape_interval, scrape_repeat_times):
    count = 1
    
    # create a tag folder
    new_folder = "ScrapeIG_" + tag
    try:
        os.mkdir(new_folder)
        os.chdir(new_folder + "/")
    except:
        os.chdir(new_folder + "/")
        
    while True:
        # set timestamp
        scrape_time = str(datetime.datetime.now().strftime("%b-%d-%Y-%H%M%S"))

        # call get_top_posts_URLs()
        # it will get URLs of the top posts in the tag and save search result page in local
        new_top_posts = get_top_posts_URLs(tag, scrape_time)
        
        # call get_post_info()
        # it will download the image and get account profile URL, number of likes and description of each top post
        profile_URLs, image_paths, top_post_likes, top_post_description = get_post_info(new_top_posts, tag, scrape_time)
        
        # call get_profile_info()
        # it will get detailed profile of each account, including name, bio and number of followers
        profile_name, profile_bio, profile_follower = get_profile_info(profile_URLs, scrape_time)
        
        # call write_info()
        # it will save all the information to database
        write_info(tag, scrape_time, new_top_posts, image_paths, top_post_likes, top_post_description, profile_URLs, profile_name, profile_bio, profile_follower)
        
        # control scrape interval and repeat_times
        count += 1
        if count <= scrape_repeat_times:
            print(f"Tag \"{tag}\" has been scraped {count-1}/{scrape_repeat_times} times. Will sleep for {scrape_interval} seconds.")
            time.sleep(scrape_interval)
        else:
            print(f"Tag \"{tag}\" has been scraped {count-1}/{scrape_repeat_times} times. Scraping finished. Saved to {os.getcwd()}.")
            os.chdir("../")
            break

# Sub Functions

### get_top_posts_URLs(tag, scrape_time)

1) get_top_posts_URLs() will get URLs to the top posts of the tag;  
  
2) It will first download the search result page and name it "tag + scrape time + search result.htm";  
  
3) It will search all top posts URLs and call existing_URLs(), add_to_existing_URLs() to check whether these posts have been scraped before, because you might see the same post in two scrapes and we don't want duplication;  
   
4) All scraped posts URLs will be saved in the text file "tag_top posts.txt";
  
5) Finally, get_top_posts_URLs() will return a list "new_top_posts", which contains all new posts URLs.  

In [4]:
def get_top_posts_URLs(tag, scrape_time):
    # URL of the search result of a tag
    URL = "https://www.instagram.com/explore/tags/{}".format(tag)
    browser.get(URL)
    
    # download the search result page
    content = browser.page_source
    with open(tag + "_" + scrape_time + "_search result.htm", mode='w', encoding='UTF-8', errors='strict', buffering=1) as f:
        f.write(content)
    
    _new_top_posts = []
    new_top_posts = []
    
    # scrape all URLs in the search result
    _all_URLs = [a.get_attribute('href') for a in browser.find_elements_by_tag_name('a')]
    
    # identify post URLs
    for URL in _all_URLs:
        if "/p/" in URL:
            _new_top_posts.append(URL)
            
    # there are 9 top posts in each search result
    _new_top_posts = _new_top_posts[0:9]
    
    # check whether the posts have been scraped
    # if not, add new URLs to the txt file and save them in new_top_posts
    try:
        for URL in _new_top_posts:
            if URL not in existing_URLs(tag, _new_top_posts):
                new_top_posts.append(URL)
        add_to_existing_URLs(tag, new_top_posts)
    except:
        pass
        
    return(new_top_posts)

### existing_URLs(tag, _new_top_posts), add_to_existing_URLs(tag, new_top_posts)

will be called in get_top_posts_URLs() to save top posts URLs

In [5]:
# read all URLs in the txt file, which contains all the scraped URLs of the tag
def existing_URLs(tag, _new_top_posts):
    try:
        with open(tag + "_top posts.txt", "r") as f:
            all_top_posts = [link.rstrip('\n') for link in f]
    except:
        all_top_posts = []
    return(all_top_posts)


# add new URLs to the txt file
def add_to_existing_URLs(tag, new_top_posts):
    with open(tag + "_top posts.txt", "a") as f:
        for link in new_top_posts:
            f.write('%s\n' % link)

### get_post_info(new_top_posts, tag, scrape_time)

1) get_post_info() takes the list of top posts URLs from get_top_posts_URLs();  

2) It will get the account profile URLs;  
  
3) It will call download_image() to download the image and webpage of each top post and return local path of image;   
  
4) It will get number of likes and post description;
  
5) Finally, get_post_info() will return four lists: profile_URLs, image_paths, top_post_likes, top_post_description.  

In [6]:
def get_post_info(new_top_posts, tag, scrape_time):
    profile_URLs = []
    image_paths = []
    top_post_likes = []
    top_post_description = []
    
    for URL in new_top_posts:
        browser.get(URL)
        source = browser.page_source
        data=bs(source, 'html.parser')
        
        # get account profile URL
        _links = [a.get_attribute('href') for a in browser.find_elements_by_tag_name('a')] 
        profile_URLs.append(_links[0])
        
        # call download_image()
        # it will download the image and webpage of each top post and return local path of image
        image_paths.append(download_image(URL, tag, scrape_time))

        # get number of likes
        try:
            like = data.find('div', class_ = 'Nm9Fw').button.span.text
            top_post_likes.append(like)
        except:
            top_post_likes.append(None)
            pass

        # get post description
        try:
            desc = data.find('div', class_ = 'C4VMK').span.text
            top_post_description.append(desc)
        except:
            top_post_description.append(None)
            pass
        
        # pause for 2 seconds
        time.sleep(2)
        
    return(profile_URLs, image_paths, top_post_likes, top_post_description)

### download_image(URL, tag, scrape_time)

Will be called in get_post_info() to download the image and webpage of each top post and return local path of image;  
Image will be the highest resolution version and saved as "tag + scrape time + suffix of post URL.jpg". Videos will be ignored

In [7]:
def download_image(URL, tag, scrape_time):
    try:
        # find the image URL
        img_links = [a.get_attribute('srcset') for a in browser.find_elements_by_tag_name('img')]
        _top_pic = img_links[1]
        _top_pic = re.findall(r"(https.*?) [0-9]+w", _top_pic)[-1]
        _top_pic = requests.get(_top_pic)
        
        # find the suffix of image URL
        suffix = re.search(r"/p/(.*)/", URL).group(1)
        
        # set filename
        filename = tag + "_" + scrape_time + "_" + suffix
        
        # download the image
        open(filename + '.jpg', 'wb').write(_top_pic.content)
        
        # download the post page
        content = browser.page_source
        with open(filename + ".htm", mode='w', encoding='UTF-8', errors='strict', buffering=1) as f:
            f.write(content)
            
        return('../ScrapeIG_' + tag + '/'+ filename + '.jpg')
    
    except:
        return("")

### get_profile_info(profile_URLs, scrape_time)

1) get_profile_info() takes profile_URLs and look into each account profile;  
   
2) It will identify the the profile name, profile bio, number of followers;  
  
3) It will download the webpage of profile and save it as "profile + scrape time + name.htm";
     
4) Finally, get_profile_info() will return them via three lists: profile_name, profile_bio, profile_follower. 

In [8]:
def get_profile_info(profile_URLs, scrape_time):
    profile_name = []
    profile_follower = []
    profile_bio = []
    
    for URL in profile_URLs:
        browser.get(URL)
        content = browser.page_source
        data=bs(content, 'html.parser')
     
        # get profile name
        try:
            name = data.find('h1', {"class": "rhpdm"}).text
            profile_name.append(name)
        except:
            profile_name.append(None)
            pass
        
        # get profile bio
        try:
            bio = "\n".join([el.text for el in data.find('div', {"class": "-vDIg"}).contents if isinstance(el, Tag)])
            profile_bio.append(bio)
        except:
            profile_bio.append(None)
            pass
        
        # get number of followers
        try:
            follower = data.find('span', {"class": "g47SY", "title" : True}).get("title")
            profile_follower.append(follower)
        except:
            profile_follower.append(None)
            pass
        
        # download the profile page
        try:
            with open("profile_" + scrape_time + "_" + name + ".htm", mode='w', encoding='UTF-8', errors='strict', buffering=1) as f:
                f.write(content)
        except:
            pass
            
        # pause for 2 seconds
        time.sleep(2)

    return(profile_name, profile_bio, profile_follower)

### write_info(tag, scrape_time, new_top_posts, image_paths, top_post_likes, top_post_description, profile_URLs, profile_name, profile_bio, profile_follower)

1) write_info() will store all the top post information into MongoDB database;  
  
2) Images downloaded in local will be stored as Binary in the database.

In [9]:
def write_info(tag, scrape_time, new_top_posts, image_paths, top_post_likes, top_post_description, profile_URLs, profile_name, profile_bio, profile_follower):
    try:
        # concat all lists to a dataframe
        df = pd.DataFrame(zip(new_top_posts, image_paths, top_post_likes, top_post_description, profile_URLs, profile_name, profile_bio, profile_follower))
        df.columns = (['new_top_posts', 'image_paths', 'top_post_likes', 'top_post_description', 'profile_URLs', 'profile_name', 'profile_bio', 'profile_follower'])
        df['tag'] = tag
        df['scrape_time'] = scrape_time


        # read local images and convert them to Binary
        df['binary_images'] = ''

        for i in range(len(df)):
            try:
                with open(df['image_paths'][i], 'rb') as img:
                    df['binary_images'][i] = Binary(img.read())
            except:
                pass


        # change columns order
        df = df[['tag', 'scrape_time', 'new_top_posts', 'image_paths', 'top_post_likes', 'top_post_description', 'profile_URLs', 'profile_name', 'profile_bio', 'profile_follower', 'binary_images']]


        # save to MongoDB
        client = MongoClient()
        db = client["ScrapeIG"]
        collection = db[tag]

        x = collection.insert_many(df.to_dict('records'))
        print("The scraping results have been saved to database:")
        print(x.inserted_ids)
        
    except:
        print("No new top post of this tag detected")

# Let's start our Instagram top posts scraping

In [17]:
# For quicker demonstration, here we only scrape twice and pause 60 seconds between scraping
scrape_tag('corgi', 60, 2)

The scraping results have been saved to database:
[ObjectId('5e6ee5cda46aec53f5145ab1'), ObjectId('5e6ee5cda46aec53f5145ab2'), ObjectId('5e6ee5cda46aec53f5145ab3'), ObjectId('5e6ee5cda46aec53f5145ab4'), ObjectId('5e6ee5cda46aec53f5145ab5'), ObjectId('5e6ee5cda46aec53f5145ab6'), ObjectId('5e6ee5cda46aec53f5145ab7'), ObjectId('5e6ee5cda46aec53f5145ab8'), ObjectId('5e6ee5cda46aec53f5145ab9')]
Tag "corgi" has been scraped 1/2 times. Will sleep for 60 seconds.
No new top post of this tag detected
Tag "corgi" has been scraped 2/2 times. Scraping finished. Saved to C:\Users\Percy\Downloads\ScrapeIG_corgi\ScrapeIG_corgi.


## We have scarped following seven tags

In [10]:
# scrape_tag('corgi', 1800, 6)

The scraping results have been saved to database:
[ObjectId('5e6e68d5a46aec53f5145aa3'), ObjectId('5e6e68d5a46aec53f5145aa4'), ObjectId('5e6e68d5a46aec53f5145aa5'), ObjectId('5e6e68d5a46aec53f5145aa6'), ObjectId('5e6e68d5a46aec53f5145aa7')]
Tag "corgi" has been scraped 1/6 times. Will sleep for 1800 seconds.
The scraping results have been saved to database:
[ObjectId('5e6e6fe7a46aec53f5145aa9')]
Tag "corgi" has been scraped 2/6 times. Will sleep for 1800 seconds.
The scraping results have been saved to database:
[ObjectId('5e6e76faa46aec53f5145aab')]
Tag "corgi" has been scraped 3/6 times. Will sleep for 1800 seconds.
The scraping results have been saved to database:
[ObjectId('5e6e7e0ba46aec53f5145aad')]
Tag "corgi" has been scraped 4/6 times. Will sleep for 1800 seconds.
No new top post of this tag detected
Tag "corgi" has been scraped 5/6 times. Will sleep for 1800 seconds.
The scraping results have been saved to database:
[ObjectId('5e6e8c28a46aec53f5145aaf')]
Tag "corgi" has been 

In [ ]:
# scrape_tag('sanfrancisco', 1800, 6)

In [ ]:
# scrape_tag('life', 1800, 6)

In [ ]:
# scrape_tag('love', 1800, 6)

In [ ]:
# scrape_tag('instagood', 1800, 6)

In [ ]:
# scrape_tag('smile', 1800, 6)

In [ ]:
# scrape_tag('cute', 1800, 6)